# 제2 고지 : 자연스러운  코드로 
## STEP 16 : 복잡한 계산 그래프(구현 편)

15단계에서 설명한 이론을 바탕으로 코드를 구현한다. 

![image](../assets/%EA%B7%B8%EB%A6%BC%2016-1.png)

### 16.1 세대 추가
먼저 **순전파**시 `세대` 를 추가하는 방법부터 살펴본다. 이를 위해, `Variable` 과 `Function`에 `generation` 을 추가하는 밥법을 순차적으로 살펴본다.

1. `Variable` 클래스 

![image](../assets/%EA%B7%B8%EB%A6%BC%2016-1.png)

- `Variable` 클래스는 `generation` 을 **0으로 초기화**한다.
- `set_creator()` 메서드가 호출될때, **부모 함수의 세대보다 1만큼 큰 값**을 설정한다.




2. `Function` 클래스

![image](../assets/%EA%B7%B8%EB%A6%BC%2016-2.png)
- `Function` 클래스의 `generation`은 **입력변수와 같은 값으로 설정**한다.
- 만약 **입력변수가 둘 이상**이라면 **가장 큰** `generation`의 수를 선택한다. 

In [6]:
import numpy as np
class Variable:
    def __init__(self, data: np.ndarray) -> None:
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")
        self.data = data
        self.grad = None  # gradient
        self.creator = None  # creator
        ##########################
        self.generation = 0 # 세대수를 기록하는 변수 
        ##########################

    def set_creator(self, func) -> None:
        self.creator = func
        ##########################
        self.generation = func.generation+1 # 세대를 기록한다 ( 부모세대 + 1)
        ########################## 

    def backward(self):
        """
        자동 역전파 (반복)
        """
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()  
            gys = [output.grad for output in f.outputs] # 1. 순전파의 결과가 **여러개의 출력인 경우**를 처리 
            gxs = f.backward(*gys) # 2. 역전파 기준 **여러 개의 입력(=순전파의 여러 개 출력)** 을 처리.
            if not isinstance(gxs,tuple): # 3. 역전파 **결과값이 하나인 경우(=역전파의 출력이 1개인 경우)** 튜플로 변환.
                gxs = (gxs,)
            for x,gx in zip(f.inputs,gxs): # 4. **역전파 결과가 여러개의 출력인 경우** 각각 대응
                #  첫 grad를 설정시에는 `그대로` 출력하고, 
                if x.grad is None : 
                    x.grad = gx 
                # 다음 미분은 기존 미분 값에 `더해준다.`
                else :
                    ## NOTE :  in-place 연산 (x.grad+=gx) 을 하지 않는 이유는 **메모리 참조**로 원하지 않는 값 변동이 일어 날 수 있다.
                    x.grad = x.grad + gx 
                
                if x.creator is not None:
                    funcs.append(x.creator)  # 하나 앞의 함수를 리스트에 추가한다

def as_array(x):

    """
    0차원 ndarray / ndarray가 아닌 경우
    """
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    """
    Function Base Class

    """

    def __call__(self, *inputs):  #  1. * 를  활용하여 임의 개수의 인수
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) # 1. 리스트 언팩
        if not isinstance(ys,tuple): # 2. 튜플이 아닌 경우 추가 지원 
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        ####################################
        self.generation = max([x.generation for x in inputs]) # Function의 generation 설정
        ####################################

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs

        # 2. 리스트의 원소가  하나라면  첫번째 원소를 반환
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        """
        구체적인 함수 계산 담당
        """
        raise NotImplementedError()

    def backward(self, gys):
        """
        역전파
        """
        raise NotImplementedError()

class Add(Function):
    def forward(self, x0,x1):
        y = x0 + x1
        return y
    def backward(self, gy):
        # 역전파시 , 입력이 1개 , 출력이 2개 
        return gy,gy 
    
def add(x0,x1):
    return Add()(x0,x1)

class Square(Function):
    def forward(self, x):
        y= x**2 
        return y 
    def backward(self, gy):
        x = self.inputs[0].data  #  수정 전 : x= self.input.data
        gx = 2 * x * gy 
        return gx 
    
def square(x):
    return Square()(x)


### 16.2 세대 순으로 꺼내기

<p align='center'>
    <img src='../assets/%EA%B7%B8%EB%A6%BC%2016-3.png' align='center' width='50%'>
    <figcaption align='center'>세대 개념이 반영된 계산 그래프</figcaption>
</p>

위의 그림을 보면, 함수 A,B,C,D의 세대는 차례로 0,1,1,2 인 것을 확인 할 수 있다. 만약 역전파를 진행한다면, **A보다 B,C의 세대가 먼저 꺼내진다.(B,C의 세대가 더 우선되므로)**

이 개념을 바탕으로 간단한 예제를 살펴보자. 
1. 더미함수를 이용하여 각 함수의 세대를 설정한다. 
2. 세대 순 오름차순 정렬을 이용해서 가장 큰 세대의 함수를 꺼낸다.

In [7]:
# 1. 더미 함수를 이용하여 세대를 설정한다.
generations = [2,0,1,4,2]
funcs = [] 

for g in generations:
    f= Function()
    f.generation = g 
    funcs.append(f)
[f.generation for f in funcs]

[2, 0, 1, 4, 2]

In [8]:
# 2. 세대 순 정렬을 이용해서 가장 큰 세대의 함수를 꺼낸다. 
funcs.sort(key=lambda x : x.generation) # 오름차순 정렬
print(f"오름 차순 정렬된 세대 : {[f.generation for f in funcs]}")
f = funcs.pop() 
print(f"가장 큰 세대 : {f.generation}")

오름 차순 정렬된 세대 : [0, 1, 2, 2, 4]
가장 큰 세대 : 4


### 16.3 Variable 클래스의 backward

가장 큰 변화는 새로 추가된 `add_func()` 함수이다. 지금까지 DeZero함수를 리스트에 추가할 때 `func.append(f)`를 호출했는데, `add_func()` 을 호출하도록 변경했다.  
이는 **DeZero 함수  리스트를 세대순으로 정렬가능**하게 하며, 그 결과 `func.pop()`시 **세대가 가장 큰 함수를 꺼내게 된다.**

참고로, 해당 함수를 구현시 **nested function(중첩 함수)** 로 구현했는데, 이는 다음 두 조건을 만족할 시 적합하다.

1. 감싸는 메서드(`backward()`) 안에서만 이용한다.
2. 감싸는 메서드(`backward()`) 에 정의된 변수 (`funcs`과 `seen_set`) 를 사용해야 한다. 

In [9]:
class Variable:
    def __init__(self, data: np.ndarray) -> None:
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")
        self.data = data
        self.grad = None  # gradient
        self.creator = None  # creator
        ##########################
        self.generation = 0 # 세대수를 기록하는 변수 
        ##########################

    def set_creator(self, func) -> None:
        self.creator = func
        ##########################
        self.generation = func.generation+1 # 세대를 기록한다 ( 부모세대 + 1)
        ########################## 

    def backward(self):
        """
        자동 역전파 (반복)
        """
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        #########################################
        funcs = [] 
        seen_set=set() # 함수의 중복 추가를 막기위해 set 으로 정의

        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x : x.generation)
                
        add_func(self.creator)
        
        #########################################
        
        while funcs:
            f = funcs.pop()  
            gys = [output.grad for output in f.outputs] # 1. 순전파의 결과가 **여러개의 출력인 경우**를 처리 
            gxs = f.backward(*gys) # 2. 역전파 기준 **여러 개의 입력(=순전파의 여러 개 출력)** 을 처리.
            if not isinstance(gxs,tuple): # 3. 역전파 **결과값이 하나인 경우(=역전파의 출력이 1개인 경우)** 튜플로 변환.
                gxs = (gxs,)
            for x,gx in zip(f.inputs,gxs): # 4. **역전파 결과가 여러개의 출력인 경우** 각각 대응
                #  첫 grad를 설정시에는 `그대로` 출력하고, 
                if x.grad is None : 
                    x.grad = gx 
                # 다음 미분은 기존 미분 값에 `더해준다.`
                else :
                    ## NOTE :  in-place 연산 (x.grad+=gx) 을 하지 않는 이유는 **메모리 참조**로 원하지 않는 값 변동이 일어 날 수 있다.
                    x.grad = x.grad + gx 
                
                if x.creator is not None:
                    #####################################################
                    # 수정 전 : funcs.append(x.creator)  # 하나 앞의 함수를 리스트에 추가한다
                    add_func(x.creator)
                    #####################################################

### 16.4 동작 확인

![image](../assets/%EA%B7%B8%EB%A6%BC%2016-4.png)

이제 세대가 큰 함수부터 꺼낼 수 있게 되었으므로, 위의 계산그래프($f(x) = y = (x^2)^2 + (x^2)^2 $)의 동작을 확인해보자. 

\begin{aligned}
&\Rightarrow f(2) = 16 + 16=32 \\ 
&\Rightarrow \frac{dy}{dx}  = 4x^3+4x^3 = 8x^3, f'(2)=64
\end{aligned}
$$

In [10]:
x = Variable(np.array(2.0))
a = square(x)
y= add(square(a),square(a))
y.backward()

print(y.data)
print(x.grad)

32.0
64.0


이번 개선을 통해 더 복잡한 계산 그래프도 다룰 수 있게 되었으며, 다음과 같이 복잡한 **연결** 도 제대로 미분할 수 있다.

<img src='../assets/%EA%B7%B8%EB%A6%BC%2016-5.png' align='center' width='50%' height='50%'>